In [ ]:
#adapted from Kaggle (simple LSTM for text classification)
#https://www.kaggle.com/kredy10/simple-lstm-for-text-classification

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Input, Embedding, Bidirectional, Conv1D, MaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
# read in data
df = pd.read_csv("small_fic_dataset.tsv",delimiter='\t',encoding='latin-1')
df.head()
df.info()
sns.countplot(df.MAINTAG)
plt.xlabel('Label')
plt.title('Number of fics in each rating')
plt.show()

In [ ]:
#get input and label vectors
X = df.TEXT
print(X.shape)
Y = df.MAINTAG
print(X.shape)
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
print(Y.shape)

In [ ]:
#split into training and test sets
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)
Y_train = to_categorical(Y_train, num_classes=4)
Y_test = to_categorical(Y_test, num_classes=4)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# parameters
max_len = 220
tok = Tokenizer(lower=True) # default lower case = true
tok.fit_on_texts(X_train)  # 
max_words = len(tok.word_index.items()) + 1 # vocal size, all words from all these stories? seems very small
print(max_words)

In [ ]:
# get glove embeddings
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
embedding_matrix = np.zeros((max_words,100))

for word,i in tok.word_index.items():
    print(word)
    print(i)
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# makes padded sequences so all inputs are of the same length
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
# try a simple CNN with max pooling
def CNN():
 
    inputs = Input(name='inputs',shape=[max_len], dtype='int32') # do we need dtype = 'int32'?
    embedded_sequences = Embedding(max_words,100,weights=[embedding_matrix],input_length=max_len)(inputs)
    layer = Conv1D(32, 5, activation='relu')(embedded_sequences)
    layer = MaxPooling1D(5)(layer)
    #layer = Conv1D(32, 5, activation='relu')(layer)
    #layer = MaxPooling1D(5)(layer)
    layer = Conv1D(32, 5, activation='relu')(layer)
    layer = MaxPooling1D(1)(layer)  # global max pooling
    layer = Flatten()(layer)
    layer = Dense(32, activation='relu')(layer)
    preds = Dense(4, activation='softmax')(layer)
    
    model = Model(inputs, preds)
    return model
    

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len]) # main input _ story texts
    # use glove here
    layer = Embedding(max_words,100,weights=[embedding_matrix],input_length=max_len)(inputs)
    layer = Bidirectional(LSTM(64))(layer)
    
    # lstm_out = 
    # add other input? auxiliary_input = Input(shape=(..,), name='aux_input')
    # x = keras.layers.concatenate([lstm_out, auxiliary_input])
    
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
   
    layer = Activation('softmax')(layer) # add other inputs before this 
    model = Model(inputs=inputs,outputs=layer)
    
    # model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])
    return model


In [ ]:
# run RNN model with LSTM
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#trains model
model.fit(sequences_matrix,Y_train,epochs=6,validation_split=0.3,shuffle=True)


In [ ]:
# test model on test set and print accuracy
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#plot_confusion_matrix(Y_test, pred_test)